In [5]:
import os
import gdown
import zipfile
from modules import logger
from src.utils.common import get_size

google drive test data link

In [4]:
# url = "https://drive.google.com/file/d/1-6haawpnaPTHxclM7WSrxhiO-Vwx2hRG/view?usp=drive_link"

In [6]:
# type(url)

str

In [8]:
# url.split("/")

['https:',
 '',
 'drive.google.com',
 'file',
 'd',
 '1-6haawpnaPTHxclM7WSrxhiO-Vwx2hRG',
 'view?usp=drive_link']

In [10]:
# file_id = url.split("/")[-2]
# file_id

'1-6haawpnaPTHxclM7WSrxhiO-Vwx2hRG'

In [11]:
# prefix = "https://drive.google.com/uc?/export=download&id=" + file_id
# gdown.download(prefix, "test_data_set.zip")

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1-6haawpnaPTHxclM7WSrxhiO-Vwx2hRG
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1-6haawpnaPTHxclM7WSrxhiO-Vwx2hRG&confirm=t&uuid=1547d607-51d0-4abc-b259-f7786c27a7ba
To: e:\Oslo\OsloMet\Fourth semester\Electoral_Symbols_And_Vote_Detection_MLOPS\Electoral_Symbols_And_Vote_Detection\research\test_data_set.zip
100%|██████████| 100M/100M [00:13<00:00, 7.19MB/s] 


'test_data_set.zip'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Oslo\\OsloMet\\Fourth semester\\Electoral_Symbols_And_Vote_Detection_MLOPS\\Electoral_Symbols_And_Vote_Detection'

In [6]:
## Configuration entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from modules.symbol_detection.faster_rcnn.constants import *
from src.utils.common import read_yaml,create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath =  CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
            
            )
        
        return data_ingestion_config

        

In [11]:

#components
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config=config

    
    def download_file(self)-> str:
        """
            Fetech data from the url
        """

        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id=" + file_id
            gdown.download(prefix, zip_download_dir)
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        
        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None

        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            


In [10]:
#pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e 

[2024-11-10 13:12:07,555: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-10 13:12:07,557: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-10 13:12:07,558: INFO: common: created directory at: artifacts]
[2024-11-10 13:12:07,559: INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-10 13:12:07,561: INFO: 1728023487: Downloading data from https://drive.google.com/file/d/1-6haawpnaPTHxclM7WSrxhiO-Vwx2hRG/view?usp=drive_link into file artifacts/data_ingestion/test_data_set.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1-6haawpnaPTHxclM7WSrxhiO-Vwx2hRG
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1-6haawpnaPTHxclM7WSrxhiO-Vwx2hRG&confirm=t&uuid=b5a09cb8-e96a-43b8-9082-099fa59574f0
To: e:\Oslo\OsloMet\Fourth semester\Electoral_Symbols_And_Vote_Detection_MLOPS\Electoral_Symbols_And_Vote_Detection\artifacts\data_ingestion\test_data_set.zip
100%|██████████| 100M/100M [00:11<00:00, 8.97MB/s] 

[2024-11-10 13:12:22,783: INFO: 1728023487: Downloaded data from https://drive.google.com/file/d/1-6haawpnaPTHxclM7WSrxhiO-Vwx2hRG/view?usp=drive_link into file artifacts/data_ingestion/test_data_set.zip]
